# ECIFEELINGS
  
### Proyecto de Inteligencia Artificial para la detección temprana de indicios de depresión en textos

**Presentado por:**  
Julian Camilo López Barrero  

**Institución:**  
Escuela Colombiana de Ingeniería Julio Garavito  

**Asignatura:**  
Proyecto de Tecnologías de Inteligencia Artificial (PTIA)

**Año:** 2025


In [34]:
# BLOQUE INSTALACIÓN E IMPORTACIONES
!pip install spacy -q
!python -m spacy download es_core_news_sm -q

import pandas as pd
import numpy as np
import re
import spacy
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Cargar modelo de lenguaje en español
nlp = spacy.load("es_core_news_sm")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 70.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [35]:
# CARGA DE LOS DATOS CON RUIDO
import random

# 1. Carga de datos DataSet Keaggle
df_depresion = pd.read_csv('spanish_tweets_suggesting_signs_of_depression_v1.csv')
df_depresion = df_depresion[['TWEET_TEXT']].rename(columns={'TWEET_TEXT': 'texto'})
df_depresion['label'] = 1

# 2. Datos Sinteticos
textos_normales_base = [
    "Hoy es un día maravilloso para salir.", "Me encanta el café de la mañana.",
    "Estoy muy feliz por mi trabajo.", "Disfrutando de las vacaciones.",
    "El partido de fútbol estuvo increíble.", "Voy a cocinar una cena deliciosa.",
    "Qué buen clima hace hoy.", "Terminé de leer un libro excelente.",
    "Me siento motivado para el gimnasio.", "La vida tiene momentos bonitos.",
    "Me siento tranquilo y relajado.", "Mañana será un gran día.",
    "Saliendo con amigos al cine.", "Escuchando mi música favorita.",
    "Todo va a salir bien."
]

# Igualar cantidad
if len(df_depresion) > 0:
    factor = (len(df_depresion) // len(textos_normales_base)) + 1
    lista_normales = (textos_normales_base * factor)[:len(df_depresion)]
else:
    lista_normales = []

df_normal = pd.DataFrame({'texto': lista_normales, 'label': 0})

# 3. Unir todo
df = pd.concat([df_depresion, df_normal], ignore_index=True)

# INYECCIÓN DE RUIDO
porcentaje_ruido = 0.15
indices_ruido = df.sample(frac=porcentaje_ruido).index
df.loc[indices_ruido, 'label'] = 1 - df.loc[indices_ruido, 'label']

# 4. Limpieza y Mezcla final
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

def limpiar_texto(texto):
    if not isinstance(texto, str): return ""
    texto = texto.lower()
    texto = re.sub(r'http\S+|www\S+', '', texto)
    texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)
    return texto

df['texto_limpio'] = df['texto'].apply(limpiar_texto)

In [36]:

MAX_NB_WORDS = 2000
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 50 # Menos dimensiones = menos capacidad de memorizar

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['texto_limpio'].values)
X = tokenizer.texts_to_sequences(df['texto_limpio'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
Y = df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [37]:
# DEFINICION MODELO

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(SpatialDropout1D(0.5))

# 2. LSTM
model.add(LSTM(16, dropout=0.4, recurrent_dropout=0.4))

# 3. Salida
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_9             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
#ENTRENAMIENTO
print("Entrenando...")
history = model.fit(X_train, Y_train, epochs=6, batch_size=32, validation_data=(X_test, Y_test), verbose=1)


Entrenando...
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.5439 - loss: 0.6839 - val_accuracy: 0.7100 - val_loss: 0.6401
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.7301 - loss: 0.6200 - val_accuracy: 0.7850 - val_loss: 0.5174
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.8098 - loss: 0.5139 - val_accuracy: 0.8375 - val_loss: 0.4459
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.8418 - loss: 0.4398 - val_accuracy: 0.8600 - val_loss: 0.4222
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.8381 - loss: 0.4249 - val_accuracy: 0.8625 - val_loss: 0.4142
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.8507 - loss: 0.4151 - val_accuracy: 0.8650 - val_loss: 0.4130


In [39]:
#PRUEBAS MASIVAS

print("\n--- PRUEBAS ---")

def probar_frase(frase):
    procesado = limpiar_texto(frase)
    seq = tokenizer.texts_to_sequences([procesado])
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    prob = model.predict(padded)[0][0]

    # Umbral de decisión
    etiqueta = "DEPRESIÓN" if prob > 0.5 else " NORMAL   "
    print(f"[{etiqueta}] {prob:.1%} -> '{frase}'")

# LISTA DE CASOS DE PRUEBA
casos = [
    # GRUPO 1: Depresión Clara (CSV)
    "Siento un vacío enorme en el pecho que no se quita con nada.",
    "Ya no tengo fuerzas para levantarme de la cama, solo quiero dormir.",
    "La soledad me está consumiendo poco a poco.",
    "No veo ningún futuro para mí, todo es oscuridad.",
    "Me odio a mí mismo y a todo lo que hago.",
    "A veces pienso que el mundo estaría mejor sin mí.",

    # GRUPO 2: Depresión Sutil (Síntomas físicos/conductuales)
    "Llevo tres días sin comer y no tengo nada de hambre.",
    "El insomnio me está matando, son las 4am y sigo pensando.",
    "Me siento cansado todo el tiempo, incluso después de dormir 10 horas.",
    "Antes disfrutaba jugar fútbol, ahora ya no me interesa nada.",
    "Solo quiero llorar y no sé por qué.",

    #  GRUPO 3: Normal / Positivo
    "Hoy es un día increíble para salir a caminar con amigos.",
    "Me siento super motivado con mi nuevo trabajo.",
    "Qué rica estuvo la cena de anoche, me encanta cocinar.",
    "Mañana tengo examen pero estudié mucho y estoy tranquilo.",
    "El tráfico estuvo horrible pero llegué a tiempo.",
    "Voy a ver una película de comedia para relajarme.",

    #  GRUPO 4: Tristeza Normal (No patológica - Difícil para la IA)
    "Estoy triste porque mi equipo de fútbol perdió el partido.",
    "Se me rompió el celular y me da mucha rabia.",
    "Hoy estoy aburrido, no hay nada bueno en la televisión.",
    "Estoy cansado de trabajar tanto, necesito vacaciones.",

    #  GRUPO 5: Casos Tramposos (Palabras clave en contexto positivo)
    "No tengo depresión, me siento genial.",
    "Estoy luchando contra la pereza para ir al gimnasio.",
    "Morí de risa con ese chiste."
]

for c in casos:
    probar_frase(c)


--- PRUEBAS ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
[DEPRESIÓN] 91.1% -> 'Siento un vacío enorme en el pecho que no se quita con nada.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[DEPRESIÓN] 93.7% -> 'Ya no tengo fuerzas para levantarme de la cama, solo quiero dormir.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[DEPRESIÓN] 75.7% -> 'La soledad me está consumiendo poco a poco.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[DEPRESIÓN] 81.7% -> 'No veo ningún futuro para mí, todo es oscuridad.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[DEPRESIÓN] 89.5% -> 'Me odio a mí mismo y a todo lo que hago.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
[DEPRESIÓN] 86.9% -> 'A veces pienso que el mundo estaría mejor sin mí.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
[DEPRESIÓN] 89.6% -> 'Llevo tres días sin comer y no tengo nada de hambre.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
[DEPRESIÓN] 67.7% -> 'El insomnio me está matando, son las 4am y sigo pensando.'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[DEPRESIÓN] 81.7% -> 'Me siento